Dropout
- this is another method of network regularization
- involves turning on and off some neurons in a layer in the network, called dropout layer, during training. This allows some inputs to the layer to pass through, while stopping others. Neurons to be turned off are randomly selected, with a given probability during each forward pass (i.e. 30% or 50% of neurons are turned off in a forward pass, neurons selected at random for each pass)
- forces the model to make use of all its neurons. This can help with:
    1) Prevents the network from becoming to reliant on any one neuron or reliance on a single neuron for a single instance.
    2) Preventing overfitting/memorizing the data as model cannot rely on specific neurons to memorize samples 
    3) Preventing co-adaptation - which is when a neuron becomes highly dependent on the outputs of another pre-synaptic neuron. Instead of learning the underlying function, it becomes specialized to the output of one of its input neurons. This can lead to poor generalization and overfitting
    4) Helpful if training data is noisy or has other perturbations. Dropout is its own form of noise - thus helping the network better generalize to noisy data. Exposing network to different subset of neurons during training teaches it to be more resilient to noise and variations in input data
- despite "turning off" some neurons, this does not speed up training process, as the dropped-out are not truly turned off, just their outputs are zeroed

Forward Pass
- firstly - bernoulli distribution and binomial distribtion. 
    1) A bernoulli distribution is the probability of a binary outcome for a single trial, like a coin toss, where outcomes are mutually exclusive. There is either heads or tails. In this scenario, heads =1, tails = 0. Probability of heads,p = .5; probability of tails = 1 - p. Generally p can anything between 0 and 1.
    2) a binomial distribtion is like the general case of a bernoulli distribtion. In general form, we can do multiple trials for outcomes from the distribution, summing them. So bernoulli distribution is a special case of binomial distribtion where a single trial is conducted. For example if we have a binomial distribtion with number of trials = 2, and p > 0. The possible range of values is 0, 1 , and 2 from this binomial distribtion. This is because when (p>0) and it is possible, in two trials, to draw a 1 each time. So summing these up, the binomial distribtion outcome is 0. It is 0 for the same reason, if p<1. and 1 may occur by drawing 1/0 or 0/1 on the trials. In the example given, (i.e. 1 or 0, where p = .5, n_samples = 2) there is 50% of a 1 (.5 + .5), and 25% of 0 or 2 (.5 * .5).
- to create our dropout, we will use a binomial distribution with n_samples = 1, and p = share of neurons we want to drop out. In some network frameworks, p may be defined oppositely, where it is the share of neurons being kept. It is just important to be aware of which way it is. We can use numpy to create an array with individual trials in each value with our desired binomial setting. See the example 1 code below - the function creates an array which should show at least one 0 out of the five positions in the array, but this will not always be the case as it is probabilistic. So on average, there will be four 1s and one 0. See example 2, it is the basic method by which dropout will be implemented.
- just dropping half the neurons presents an issue because we will be cutting the output by approximately the share of neurons that we drop. So similar to the normalization for adam, we need to divide by the (1 - dropout_rate) to rescale the dropout outputs back to their full values. So this way, we don't have to account for the dropout during prediction, and with enough samples, this scaling should average out to the approximate amount it would be without dropout. See example 3, notice how after scaling across many samples, the dropout sum is very close to the actual sum without dropout (intial sum)


In [6]:
import numpy as np

##Example 1
dropout_rate = 0.20 #how many should be 0
print("Example 1 output: ",np.random.binomial(1, 1-dropout_rate, size=5))

##Example 2
import numpy as np
dropout_rate = 0.3
example_output = np.array([0.27, -1.03, 0.67, 0.99, 0.05, -0.37, -2.01, 1.13, -0.07, 0.73])
example_output *= np.random.binomial(1, 1-dropout_rate, example_output.shape) #final argument is called size in the function, but we need to pass shape. It's confusing.
print("Example 2 output: ",example_output)

##Example 3
print("Example 3 output")
import numpy as np
dropout_rate = 0.2
example_output = np.array([0.27, -1.03, 0.67, 0.99, 0.05, -0.37, -2.01, 1.13, -0.07, 0.73])
print(f'sum initial {sum(example_output)}')
sums = []
for i in range(10000):
    example_output2 = example_output * np.random.binomial(1, 1-dropout_rate, example_output.shape) / (1-dropout_rate)
    sums.append(sum(example_output2))
print(f'mean sum: {np.mean(sums)}')

Example 1 output:  [1 1 1 1 1]
Example 2 output:  [ 0.   -0.    0.    0.99  0.05 -0.   -0.    1.13 -0.07  0.73]
Example 3 output
sum initial 0.36000000000000015
mean sum: 0.3605887500000002


Backward Pass
- We have done two changes to the dropout layer that must be accounted for
    1) as described, during dropout, a neuron can have two states, dropped or not dropped, so the neuron's output is either the output or 0
    2) then, we have also normalized this output so that it is closer to the true sum (see discussion above).

- we'll start with the scalar derivative (dividing by (1- dropout)). At first glance it may seem like quotient rule, but actually we just have a linear equation: f(x) = 1/(1 - q)*x => f'(x) = 1/(1-q), where 1 = dropout rate. We just treat the 1/(1-q) as a constant, so the derivative becomes simple dropping the linear variable. Note, that we use this derivative for neurons that are not dropped in the dropout layer.
- for neurons that are dropped, the derivative is 0. Intuitively, this makes sense, they are not contributing at all to loss.
- Please note that the dropout itself does not have any neurons, it just goes after a given layer in the network to turn off some neurons. So that's why the derivative is 1/(1-q) for not dropped neurons => their weight/input/bias derivatives are handled in layer before the dropout layer (from a forward pass perspective).
- full dropout derivative: where droput is D(z) and ri is the 1/0 mask for dropping neurons, z is the input from previous layer, q is dropout rate; 
    - D(z) = z/(1-q) where ri=1, 0 where ri=0
    - D'(z) = 1/(1-q) where ri=1, 0 where ri=0